In [4]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [5]:
df_green = spark.read.parquet('data/pq/green/*/*')
df_green.show(3)

+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|VendorID|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|       2| 2020-01-12 18:15:04|  2020-01-12 18:19:52|                 N|         1|          41|          41|              1|         0.78|        5.5|  0.0|    0.

In [15]:
# Implement the followin query with RDDs:

In [16]:
'''
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
'''


"\nSELECT \n    date_trunc('hour', lpep_pickup_datetime) AS hour, \n    PULocationID AS zone,\n\n    SUM(total_amount) AS amount,\n    COUNT(1) AS number_records\nFROM\n    green\nWHERE\n    lpep_pickup_datetime >= '2020-01-01 00:00:00'\nGROUP BY\n    1, 2\n"

In [18]:
#Context & Use Cases:
#--------------------
#- DataFrames offer a high-level, optimized interface for structured data manipulation. 
#They are recommended for most data processing tasks due to ease of use and performance benefits.
    
#- RDDs provide a lower-level API that allows for granular control over distributed data, 
#  useful for custom computations or integrating legacy Spark code.
      
#Considerations:
#---------------
#- Transitioning to RDDs from DataFrames bypasses DataFrame optimizations but is necessary 
#  for specific scenarios requiring intricate data manipulations not supported by DataFrame operations.

#Guidance:
#---------
#- Default to DataFrames for efficiency and simplicity. Opt for RDDs when encountering 
#limitations in expressing complex logic through DataFrames.

#--> Here we will see some basic ops regarding RDDs!

In [6]:
rdd = df_green \
    .select('lpep_pickup_datetime', 'PULocationID', 'total_amount') \
    .rdd

rdd.take(5)

[Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 12, 18, 15, 4), PULocationID=41, total_amount=7.88),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 31, 20, 24, 10), PULocationID=173, total_amount=8.3),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 7, 8, 16, 53), PULocationID=74, total_amount=23.46),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 15, 14, 47, 15), PULocationID=25, total_amount=7.3),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 31, 10, 8), PULocationID=259, total_amount=25.54)]

In [7]:
rdd.filter(lambda row: False).take(1)

[]

In [21]:
from datetime import datetime

start = datetime(year=2020, month=1, day=1)

def filter_outliers(row):
    return row.lpep_pickup_datetime >= start

rdd.filter(filter_outliers).take(1)

Traceback (most recent call last):
  File "/home/Basti/spark/spark-3.3.2-bin-hadoop3/python/pyspark/serializers.py", line 458, in dumps
    return cloudpickle.dumps(obj, pickle_protocol)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/Basti/spark/spark-3.3.2-bin-hadoop3/python/pyspark/cloudpickle/cloudpickle_fast.py", line 73, in dumps
    cp.dump(obj)
  File "/home/Basti/spark/spark-3.3.2-bin-hadoop3/python/pyspark/cloudpickle/cloudpickle_fast.py", line 602, in dump
    return Pickler.dump(self, obj)
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/Basti/spark/spark-3.3.2-bin-hadoop3/python/pyspark/cloudpickle/cloudpickle_fast.py", line 692, in reducer_override
    return self._function_reduce(obj)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/Basti/spark/spark-3.3.2-bin-hadoop3/python/pyspark/cloudpickle/cloudpickle_fast.py", line 565, in _function_reduce
    return self._dynamic_function_reduce(obj)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/B

PicklingError: Could not serialize object: IndexError: tuple index out of range

In [22]:
rows = rdd.take(10)
row = rows[0]

Traceback (most recent call last):
  File "/home/Basti/spark/spark-3.3.2-bin-hadoop3/python/pyspark/serializers.py", line 458, in dumps
    return cloudpickle.dumps(obj, pickle_protocol)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/Basti/spark/spark-3.3.2-bin-hadoop3/python/pyspark/cloudpickle/cloudpickle_fast.py", line 73, in dumps
    cp.dump(obj)
  File "/home/Basti/spark/spark-3.3.2-bin-hadoop3/python/pyspark/cloudpickle/cloudpickle_fast.py", line 602, in dump
    return Pickler.dump(self, obj)
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/Basti/spark/spark-3.3.2-bin-hadoop3/python/pyspark/cloudpickle/cloudpickle_fast.py", line 692, in reducer_override
    return self._function_reduce(obj)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/Basti/spark/spark-3.3.2-bin-hadoop3/python/pyspark/cloudpickle/cloudpickle_fast.py", line 565, in _function_reduce
    return self._dynamic_function_reduce(obj)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/B

PicklingError: Could not serialize object: IndexError: tuple index out of range